### Creating CNN model

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as keras
import matplotlib.pyplot as plt
import os
import cv2 as cv

In [2]:
from sklearn.model_selection import train_test_split

rice_data=np.load('./rice_dataset.npy')
labels=np.load('./labels.npy')
x_train, x_test, y_train, y_test=train_test_split(rice_data,labels,train_size=0.8)

def accuracy(model):
    pred=model.predict(x_test)
    y_hat=np.argmax(pred,axis=1)
    y=np.argmax(y_test,axis=1)
    error=1-np.sum(np.not_equal(y_hat,y))/len(y_hat)
    return error

In [3]:
inp=keras.Input(shape=(200,200,3))
conv1=keras.layers.Conv2D(16,(3,3),(1,1),'same')
x=conv1(inp)
batchNorm1=keras.layers.BatchNormalization(momentum=0.7)
x=batchNorm1(x)
act1=keras.layers.Activation('relu')
x=act1(x)
maxPool1=keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same')
x=maxPool1(x)

conv2=keras.layers.Conv2D(32,(3,3),(1,1),'same')
x=conv2(x)
batchNorm2=keras.layers.BatchNormalization(momentum=0.7)
x=batchNorm2(x)
act2=keras.layers.Activation('relu')
x=act2(x)
maxPool2=keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same')
x=maxPool2(x)

flatten=keras.layers.Flatten()
x=flatten(x)

dense=keras.layers.Dense(300,activation='relu')
x=dense(x)
final=keras.layers.Dense(3,activation='softmax')
out=final(x)
model=keras.Model(inp,out)

In [4]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [5]:
with tf.device('/device:GPU:0'):
    model.fit(rice_data,labels, validation_data=(x_test,y_test),epochs=6,batch_size=24)

Epoch 1/6
23/23 [==============================] - 15s 378ms/step - loss: 24.3913 - categorical_accuracy: 0.4074 - val_loss: 2.6574 - val_categorical_accuracy: 0.6937
Epoch 2/6
23/23 [==============================] - 4s 167ms/step - loss: 3.7303 - categorical_accuracy: 0.6763 - val_loss: 2.4197 - val_categorical_accuracy: 0.8018
Epoch 3/6
23/23 [==============================] - 4s 178ms/step - loss: 2.9800 - categorical_accuracy: 0.7497 - val_loss: 1.0088 - val_categorical_accuracy: 0.8559
Epoch 4/6
23/23 [==============================] - 4s 175ms/step - loss: 3.8781 - categorical_accuracy: 0.7554 - val_loss: 0.3744 - val_categorical_accuracy: 0.9099
Epoch 5/6
23/23 [==============================] - 4s 180ms/step - loss: 1.1348 - categorical_accuracy: 0.8686 - val_loss: 0.2941 - val_categorical_accuracy: 0.9279
Epoch 6/6
23/23 [==============================] - 4s 174ms/step - loss: 0.3053 - categorical_accuracy: 0.9497 - val_loss: 0.0064 - val_categorical_accuracy: 1.0000


In [6]:
model.save_weights('./weights.h5',save_format='H5')
string=model.to_json()
with open('./model_config.json','w') as file:
    file.write(string)
file.close()

###### Testing model

In [7]:
model=keras.models.load_model('./CNN_model.h5')

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 200, 200, 16)      64        
_________________________________________________________________
activation (Activation)      (None, 200, 200, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 100, 100, 32)      0

### Testing Functions

In [9]:
def inp():
    global label
    loc='./rice/'
    label=os.listdir(loc)
    n=int(input(f"Enter index with respect to {label}\n"))
    req_resol=(200,200)
    items=os.listdir(loc+'/'+label[n])
    rand=np.random.randint(len(items))
    
    img=cv.imread(loc+'/'+label[n]+'/'+items[rand])
    img=cv.resize(img,req_resol,interpolation=cv.INTER_AREA)/255
    return img

In [10]:
def predict():
    image=inp()
    y_hat=model.predict(image.reshape((1,200,200,3)))
    ind=np.argmax(y_hat)
    print(label[ind])

### Results

In [11]:
while(True):
    try:
        predict()
    except:
        break

Enter index with respect to ['Bacterial leaf blight', 'Brown spot', 'Leaf smut']
0
Bacterial leaf blight
Enter index with respect to ['Bacterial leaf blight', 'Brown spot', 'Leaf smut']
1
Brown spot
Enter index with respect to ['Bacterial leaf blight', 'Brown spot', 'Leaf smut']
1
Brown spot
Enter index with respect to ['Bacterial leaf blight', 'Brown spot', 'Leaf smut']
1
Brown spot
Enter index with respect to ['Bacterial leaf blight', 'Brown spot', 'Leaf smut']
2
Leaf smut
Enter index with respect to ['Bacterial leaf blight', 'Brown spot', 'Leaf smut']

